In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:

data_path = os.path.join(os.path.dirname(os.getcwd()), 'raw_data/')
json_files = [files for files in os.listdir(data_path) if files.endswith('.json')]
print(json_files)

['london_marathon_sk.json', 'london_cycling_sk.json', 'berlin_swimmimg.json', 'berlin_hiking.json', 'london_sports_sk.json', 'london_hiking_sk.json', 'london_fitness_sk.json', 'london_kayaking_sk.json', 'berlin_cycling.json', 'berlin_running.json', 'london_skating_sk.json', 'berlin_sports.json', 'london_yoga_sk.json', 'london_swimmimg_sk.json', 'london_football_sk.json', 'london_riding_sk.json', 'berlin_canoeing.json']


In [3]:
user_df = pd.read_csv(f'{data_path}users.csv')

In [4]:
user_df.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
id,19,20,21,22,23
email,johanna@gmail.com,anjali@gmail.com,alexandra@gmail.com,ieva@gmail.com,daniel@gmail.com
encrypted_password,$2a$12$rZkuGDgbHIHpi3sYfS4or.EmNEd61QoDhi3IEEL...,$2a$12$L.9JFRFJeF6GMfrvHQ9ycuxwAOS3ddYMeuhj7Qr...,$2a$12$ZBcWOyC2RU8aP5nWLKMlee54mNk4gmRIkFZPYL7...,$2a$12$xRfA6rvHxBLt0ETNb.fGputBmPcU1tEd6KiEXIr...,$2a$12$iElgTE/LWI.UAdBDKEUxYOpdme2qXF/nQyiuM2i...
reset_password_token,NaN,NaN,NaN,NaN,NaN
reset_password_sent_at,NaN,NaN,NaN,NaN,NaN
remember_created_at,NaN,NaN,NaN,NaN,NaN
created_at,2022-05-21 10:12:18.076423,2022-05-21 10:12:18.316674,2022-05-21 10:12:18.556628,2022-05-21 10:12:18.796555,2022-05-21 10:12:19.041699
updated_at,2022-05-21 10:12:18.076423,2022-05-21 10:12:18.316674,2022-05-21 10:12:18.556628,2022-05-21 10:12:18.796555,2022-05-21 10:12:19.041699
first_name,Johanna,Anjali,Alexandra,Ieva,Daniel


In [5]:
offers_df = pd.read_csv(f'{data_path}offers.csv')

In [6]:
data_path = os.path.join(os.path.dirname(os.getcwd()), 'raw_data/')

json_files = [files for files in os.listdir(data_path) if files.endswith('.json')]
#print(json_files)
data_dict = []
for val in json_files:
    with open(data_path+val) as f:
        data = json.load(f)
         
        for val in data:
            data_dict.append({'title': val['node']['result']['title'],
                              'description': val['node']['result']['description']
                              
                        
                             })
        


In [7]:
df = pd.DataFrame.from_dict(data_dict).drop_duplicates()
df.shape

(147, 2)

In [8]:
df.duplicated().sum()

0

# simple cleaning of description

In [9]:
import re
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

In [11]:
df['description'][0].split()

['Join',
 'our',
 'social',
 'run',
 'club!',
 'Our',
 'run',
 'clubs',
 'are',
 'a',
 'place',
 'for',
 'like-minded',
 'runners',
 '(of',
 'all',
 'levels)',
 'to',
 'come',
 'together,',
 'be',
 'active,',
 'and',
 'get',
 'social.',
 'We',
 'run',
 'an',
 'open',
 'level',
 '5k',
 'together',
 'every',
 'Sunday',
 'Morning',
 'in',
 'Hyde',
 'Park,',
 'we',
 'go',
 'at',
 'the',
 'pace',
 'of',
 'the',
 'last',
 'runner,',
 'and',
 'we',
 'focus',
 'as',
 'much',
 'on',
 'the',
 'conversation',
 'as',
 'on',
 'the',
 'run',
 'itself.',
 'We',
 'began',
 'in',
 'June',
 '2021,',
 'and',
 'now',
 'have',
 'a',
 'lovely',
 'group',
 'of',
 'members,',
 'ages',
 'ranging',
 'from',
 '17-65,',
 'and',
 'running',
 'abilities',
 'ranging',
 'from',
 'beginners',
 'to',
 'ultra-marathon',
 'runners.',
 'This',
 'dynamic',
 'is',
 'so',
 'special',
 'and',
 'has',
 'brought',
 'up',
 'some',
 'lovely',
 'moments.',
 'We',
 'invite',
 'anyone',
 'and',
 'everyone',
 'to',
 'come',
 'join',


In [12]:
df['description_clean'] = df['description'].apply(lambda x: remove_punctuation(str(x)).lower().split())
#df['description_clean'] = df['description'].apply(lambda x: remove_punctuation(x))
#df['description_clean'] = df['description'].apply(lambda x: str(x).split())
#df['description'] = df['description'].apply(lambda x: re.sub('[^A-Za-z0-9] ', '', str(x)))

#df['description'] = df['description'].apply(lambda x: re.sub('^(http:+|https:)// ', '', str(x)))

In [13]:
df.description_clean

0      [join, our, social, run, club, our, run, clubs...
3      [the, ultimate, online, law, of, attractionpos...
4      [all, night, riichi, 6pm, saturday, till, 6am,...
5      [“the, cyclist’s, marathon”, –, the, ride, lon...
6      [the, ultimate, online, law, of, attractionpos...
                             ...                        
248    [outdoor, events, have, restarted, see, httpsw...
252    [canoe, ride, around, the, spreewald, river, d...
253    [canoe, ride, around, the, spreewald, river, d...
254    [canoe, ride, around, the, spreewald, river, d...
255    [canoe, ride, around, the, spreewald, river, d...
Name: description_clean, Length: 147, dtype: object

# simple tfidf model: using the raw description

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf = TfidfVectorizer(stop_words="english")

In [15]:
tfidf_matrix = tfidf.fit_transform(df["description"])
tfidf_matrix.shape

(147, 4386)

In [16]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T)

In [17]:
title_to_index = pd.Series(df.index, index = df["title"]).drop_duplicates()

In [18]:
def content_recommender(title, cosine_sim = cosine_sim, df = df, title_to_index = title_to_index):
    idx = title_to_index[title]
    
    sim_scores = list(enumerate(cosine_sim[idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:11] # top 10 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    return df["title"].iloc[sport_indices]

In [19]:
df.head(-5)

,title,description,description_clean
0,Hyde Park Social 5k Run,Join our social run club!\n\nOur run clubs are...,"[join, our, social, run, club, our, run, clubs..."
3,*SAT 4TH JUNE ONLINE MARATHON 2022!!!* Law Of ...,THE ULTIMATE ONLINE LAW OF ATTRACTION/POSITIVI...,"[the, ultimate, online, law, of, attractionpos..."
4,All Night Riichi Mahjong Marathon,All Night Riichi\n===\n\n6pm Saturday till 6am...,"[all, night, riichi, 6pm, saturday, till, 6am,..."
5,RideLondon-Essex 100,**“The cyclist’s marathon” – the Ride London i...,"[“the, cyclist’s, marathon”, –, the, ride, lon..."
6,*SAT 2ND JULY ONLINE MARATHON 2022!!!* Law Of ...,THE ULTIMATE ONLINE LAW OF ATTRACTION/POSITIVI...,"[the, ultimate, online, law, of, attractionpos..."
...,...,...,...
227,Football game. CCFC (Clapham Common Football C...,A few places spare tomorrow.\n\nMeet up player...,"[a, few, places, spare, tomorrow, meet, up, pl..."
228,PRIME Football at Powerleague Vauxhall,Football session for over 35s on the 4G astrot...,"[football, session, for, over, 35s, on, the, 4..."
230,Walking football,Just show up at the pitch in front of the Leis...,"[just, show, up, at, the, pitch, in, front, of..."
232,Real walks & cycle rides - RESTART!!,OUTDOOR EVENTS HAVE RESTARTED!!!!!!!!!!!!!!!!!...,"[outdoor, events, have, restarted, see, httpsw..."


In [21]:
content_recommender("5 km")

KeyError: '5 km'